In [5]:
import itertools
import typing as tp

import numpy as np

In [6]:
capacity_x = 8
capacity_y = 11

garbage_dict = {
    "id_1": [
        [0, 0],
        [1, 0],
        [2, 0],
        [2, 1],
    ],
    "id_2": [
        [0, 0],
        [1, 0],
        [0, 1],
        [1, 1],
    ],
    "id_3": [
        [0, 0],
        [1, 0],
        [1, 1],
    ],
}

In [7]:
area = np.zeros(capacity_x * capacity_y).reshape(capacity_y, capacity_x)
garbage_np = {id_: np.flip(coords) for id_, coords in garbage_dict.items()}

In [8]:
def index_figure(figure):
    return figure[:, 0], figure[:, 1]

In [9]:
area[index_figure(garbage_np['id_1'])] = 1
area

array([[1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
garbage_np['id_1']

array([[1, 2],
       [0, 2],
       [0, 1],
       [0, 0]])

In [11]:
def rotate_90(figure, times: int, shift):
    return figure @ np.linalg.matrix_power(np.array([
    [0, -1],
    [1, 0],
]), times).astype(np.int_) + shift

In [12]:
def fill_area(capacity_x: int, capacity_y: int, figures):
    area = np.zeros(capacity_x * capacity_y).reshape(capacity_y, capacity_x).astype(np.int_)
    for figure in figures:
        area[index_figure(figure)] = 1
    return area

In [13]:
rotate_90(garbage_np['id_1'], -3, [1, 0])
# [y, x]

array([[ 3, -1],
       [ 3,  0],
       [ 2,  0],
       [ 1,  0]])

In [14]:
figure_1 = rotate_90(garbage_np['id_1'], 0, [0, 0])
figure_2 = rotate_90(garbage_np['id_2'], 0, [1, 0])
figure_3 = rotate_90(garbage_np['id_3'], 1, [2, 2])
figures = [figure_1, figure_2, figure_3]
fill_area(capacity_x, capacity_y, figures)

array([[1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [15]:
occupancy_grid = fill_area(capacity_x, capacity_y, figures)
occupancy_grid

array([[1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
distance_map = np.zeros_like(occupancy_grid)

In [17]:
for dy in range(capacity_y):
    for dx in range(capacity_x):
        if occupancy_grid[dy, dx] == 1:
            continue
        distance_map[dy, dx] = min(distance_map[dy - 1, dx] + 1, distance_map[dy, dx - 1] + 1)

In [18]:
distance_map

array([[0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 2, 2, 2, 2],
       [0, 0, 0, 1, 2, 3, 3, 3],
       [1, 0, 0, 1, 2, 3, 4, 4],
       [1, 1, 1, 2, 3, 4, 5, 5],
       [1, 2, 2, 3, 4, 5, 6, 6],
       [1, 2, 3, 4, 5, 6, 7, 7],
       [1, 2, 3, 4, 5, 6, 7, 8],
       [1, 2, 3, 4, 5, 6, 7, 8],
       [1, 2, 3, 4, 5, 6, 7, 8],
       [1, 2, 3, 4, 5, 6, 7, 8]])

In [19]:
def move_to_corner(figure):
    return figure + [distance_map.shape[0] - np.max(figure[:, 0]) - 1, distance_map.shape[1] - np.max(figure[:, 1]) - 1]

In [20]:
# move to edge
cand = move_to_corner(garbage_np['id_1'])
cand

array([[10,  7],
       [ 9,  7],
       [ 9,  6],
       [ 9,  5]])

In [21]:
def calc_figure_cost(figure) -> int:
    coord_set = set()
    coord_set = set((coord[0], coord[1]) for coord in figure)
    # for coord in figure:
    #     coord_set.update({(coord[0] + dy, coord[1] + dx) for dy, dx in itertools.product(range(-1, 2), repeat=2)})

    cost = sum(distance_map[coord_y, coord_x] for coord_y, coord_x in coord_set)
    # cost = 0
    # for coord_y, coord_x in coord_set:
    #     if coord_y >= capacity_y or coord_x >= capacity_x:
    #         cost += 100
    #         continue
    #     cost += distance_map[coord_y, coord_x]

    return cost

In [22]:
# calc current cost
calc_figure_cost(cand)

29

In [23]:
def is_valid_position(figure) -> bool:
    occupancy_grid
    for coord_y, coord_x in figure:
        if not 0 <= coord_y < capacity_y or not 0 <= coord_x < capacity_x:
            return False
        if occupancy_grid[coord_y, coord_x] == 1:
            return False
    return True

In [24]:
shift_steps = [
    [-1, -1],
    [-1, 0],
    [0, -1],
    [1, -1],
]

optimal_step = [0, 0]
optimal_cost = calc_figure_cost(cand)
for step in shift_steps:
    possible_cand = cand + step
    if not is_valid_position(possible_cand):
        continue
    possible_cost = calc_figure_cost(possible_cand)
    if possible_cost <= optimal_cost:
        optimal_step = step
        optimal_cost = possible_cost

# if optimal_step == [0, 0]:
#     break

In [25]:
def find_optimal_pos(figure):
    optimal_cand = None
    cand = move_to_corner(figure)
    while True:
        optimal_step = [0, 0]
        optimal_cost = float('inf')
        if is_valid_position(cand):
            optimal_cost = calc_figure_cost(cand)
        for step in shift_steps:
            possible_cand = cand + step
            if not is_valid_position(possible_cand):
                continue
            possible_cost = calc_figure_cost(possible_cand)
            if possible_cost <= optimal_cost:
                optimal_step = step
                optimal_cost = possible_cost

        if optimal_step == [0, 0]:
            break
        cand += optimal_step

    if optimal_cost != float('inf'):
        optimal_cand = cand

    return optimal_cand, optimal_cost

In [26]:
find_optimal_pos(garbage_np['id_1'])

(array([[5, 2],
        [4, 2],
        [4, 1],
        [4, 0]]),
 5)

In [27]:
init_cand = garbage_np['id_1']

global_optimal_cand = None
global_optimal_cost = float('inf')
for rotate in range(4):
    cand = rotate_90(init_cand, rotate, [0, 0])
    local_optimal_cand, local_optimal_cost = find_optimal_pos(cand)

    if local_optimal_cand is None:
        continue
    if local_optimal_cost < global_optimal_cost:
        global_optimal_cand = local_optimal_cand
        global_optimal_cost = local_optimal_cost

global_optimal_cand, global_optimal_cost

(array([[3, 0],
        [4, 0],
        [4, 1],
        [4, 2]]),
 4)

In [28]:
occupancy_grid

array([[1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [29]:
global_optimal_cand.max(axis=0), global_optimal_cand.min(axis=0)

(array([4, 2]), array([3, 0]))

In [30]:
np.prod(global_optimal_cand.max(axis=0) - global_optimal_cand.min(axis=0) + 1)

6

In [33]:
occupancy_grid

array([[1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [48]:
shape_x = 4
shape_y = 4

for dy in range(capacity_y - 1, shape_y - 1, -1):
    for dx in range(capacity_x - 1, shape_x - 1, -1):
        if not np.any(occupancy_grid[dy-shape_y:dy, dx-shape_x:dx]):
            print(True)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
